<a href="https://colab.research.google.com/github/DucBox/Sentiment-Analysis-using-Logistic-Regression/blob/main/Sentiment_Analysis_using_Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# run this cell to import nltk
import nltk
from os import getcwd
import w1_unittest

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
filePath = f"{getcwd()}/../tmp2/"
nltk.data.path.append(filePath)

In [10]:
import numpy as np
import pandas as pd
from nltk.corpus import twitter_samples

from utils import process_tweet, build_freqs

In [11]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

In [12]:
# split the data into two pieces, one for training and one for testing (validation set)
test_pos = all_positive_tweets[4000:]
train_pos = all_positive_tweets[:4000]
test_neg = all_negative_tweets[4000:]
train_neg = all_negative_tweets[:4000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

In [15]:
# combine positive and negative labels
train_y = np.append(np.ones((len(train_pos), 1)), np.zeros((len(train_neg), 1)), axis=0)
test_y = np.append(np.ones((len(test_pos), 1)), np.zeros((len(test_neg), 1)), axis=0)

In [16]:
# Print the shape train and test sets
print("train_y.shape = " + str(train_y.shape))
print("test_y.shape = " + str(test_y.shape))

train_y.shape = (8000, 1)
test_y.shape = (2000, 1)


In [13]:
print(train_neg[1000])

@seanactual You mean you're not offering? :(


In [17]:
print(train_y[1000])

[1.]


In [18]:
# create frequency dictionary
freqs = build_freqs(train_x, train_y)

# check the output
print("type(freqs) = " + str(type(freqs)))
print("len(freqs) = " + str(len(freqs.keys())))

type(freqs) = <class 'dict'>
len(freqs) = 11427


In [19]:
# test the function below
print('This is an example of a positive tweet: \n', train_x[0])
print('\nThis is an example of the processed version of the tweet: \n', process_tweet(train_x[0]))

This is an example of a positive tweet: 
 #FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)

This is an example of the processed version of the tweet: 
 ['followfriday', 'top', 'engag', 'member', 'commun', 'week', ':)']


In [20]:
def sigmoid(z):
    '''
    Input:
        z: is the input (can be a scalar or an array)
    Output:
        h: the sigmoid of z
    '''

    ### START CODE HERE ###
    # calculate the sigmoid of z
    h = 1/(1 + np.exp(-z))
    ### END CODE HERE ###

    return h

In [22]:
def gradientDescent(x, y, theta, alpha, num_iters):
    '''
    Input:
        x: matrix of features which is (m,n+1)
        y: corresponding labels of the input matrix x, dimensions (m,1)
        theta: weight vector of dimension (n+1,1)
        alpha: learning rate
        num_iters: number of iterations you want to train your model for
    Output:
        J: the final cost
        theta: your final weight vector
    '''
    m = len(x)  # number of training examples

    for i in range(num_iters):
        # Get z, the dot product of x and theta
        z = np.dot(x, theta)

        # Get the sigmoid of z
        h = 1 / (1 + np.exp(-z))

        # Calculate the cost function
        J = (-1/m) * (np.dot(y.T, np.log(h)) + np.dot((1 - y).T, np.log(1 - h)))

        # Update the weights theta
        theta = theta - (alpha/m) * np.dot(x.T, (h - y))

        # Optionally print the cost every 100 iterations
        if i % 100 == 0:
            print(f"Iteration {i} - Cost: {J[0]}")

    J = float(J)
    return J, theta

In [24]:
def extract_features(tweet, freqs, process_tweet=process_tweet):
    '''
    Input:
        tweet: a string containing one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output:
        x: a feature vector of dimension (1,3)
    '''
    # process_tweet tokenizes, stems, and removes stopwords
    word_l = process_tweet(tweet)

    # 3 elements for [bias, positive, negative] counts
    x = np.zeros(3)

    # bias term is set to 1
    x[0] = 1

    ### START CODE HERE ###

    # loop through each word in the list of words
    for word in word_l:

      # increment the word count for the positive label 1
        x[1] += freqs.get((word, 1), 0)

        # increment the word count for the negative label 0
        x[2] += freqs.get((word, 0), 0)

    ### END CODE HERE ###

    x = x[None, :]  # adding batch dimension for further processing
    assert(x.shape == (1, 3))
    return x

In [49]:
# collect the features 'x' and stack them into a matrix 'X'
X = np.zeros((len(train_x), 3))
for i in range(len(train_x)):
    X[i, :]= extract_features(train_x[i], freqs)

# training labels corresponding to X
Y = train_y

# Apply gradient descent
J, theta = gradientDescent(X, Y, np.zeros((3, 1)), 1e-9, 10000)
print(f"The cost after training is {J:.8f}.")
print(f"The resulting vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")

import pickle

# Giả sử theta là vector trọng số của bạn sau khi huấn luyện
model = {
    "theta": theta,
    "freqs": freqs,  # lưu các thông số khác nếu cần, ví dụ như freqs nếu bạn dùng nó trong extract_features
}

# Lưu mô hình vào file
with open('logistic_model.pkl', 'wb') as file:
    pickle.dump(model, file)

print("Model saved successfully.")

Iteration 0 - Cost: [0.69314718]
Iteration 100 - Cost: [0.59522326]
Iteration 200 - Cost: [0.52186107]
Iteration 300 - Cost: [0.4654039]
Iteration 400 - Cost: [0.42087709]
Iteration 500 - Cost: [0.38501586]
Iteration 600 - Cost: [0.35561279]
Iteration 700 - Cost: [0.33113124]
Iteration 800 - Cost: [0.31047349]
Iteration 900 - Cost: [0.29283724]
Iteration 1000 - Cost: [0.2776243]
Iteration 1100 - Cost: [0.26438071]
Iteration 1200 - Cost: [0.25275648]
Iteration 1300 - Cost: [0.24247801]
Iteration 1400 - Cost: [0.23332866]
Iteration 1500 - Cost: [0.22513495]
Iteration 1600 - Cost: [0.21775647]
Iteration 1700 - Cost: [0.21107852]
Iteration 1800 - Cost: [0.20500656]
Iteration 1900 - Cost: [0.19946205]
Iteration 2000 - Cost: [0.19437923]
Iteration 2100 - Cost: [0.18970269]
Iteration 2200 - Cost: [0.18538542]
Iteration 2300 - Cost: [0.1813873]
Iteration 2400 - Cost: [0.17767392]
Iteration 2500 - Cost: [0.17421556]
Iteration 2600 - Cost: [0.17098647]
Iteration 2700 - Cost: [0.16796425]
Iterati

<ipython-input-22-f25ee51c6219>:32: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  J = float(J)


In [43]:
def predict_tweet(tweet, freqs, theta):
    '''
    Input:
        tweet: a string
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
        theta: (3,1) vector of weights
    Output:
        y_pred: the probability of a tweet being positive or negative
    '''
    ### START CODE HERE ###

    # extract the features of the tweet and store it into x
    x = extract_features(tweet, freqs)
    # make the prediction using x and theta
    z = np.dot(x, theta)
    y_pred = 1 / (1 + np.exp(-z))

    ### END CODE HERE ###

    return y_pred

In [44]:
# Run this cell to test your function
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


I am happy -> 0.553817
I am bad -> 0.488441
this movie should have been great. -> 0.545885
great -> 0.544901
great great -> 0.589083
great great great -> 0.631873
great great great great -> 0.672684


<ipython-input-44-4fb6e5b5a511>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  print( '%s -> %f' % (tweet, predict_tweet(tweet, freqs, theta)))


In [45]:
# Feel free to change the tweet below
my_tweet = 'I am very disappointed with the product quality. It broke after just one use.'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs, theta)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else:
    print('Negative sentiment')

['disappoint', 'product', 'qualiti', 'broke', 'one', 'use']
[[0.49889379]]
Negative sentiment


In [50]:
import pickle

# Tải mô hình từ file
with open('/content/logistic_model.pkl', 'rb') as file:
    loaded_model = pickle.load(file)

theta1 = loaded_model["theta"]
freqs1 = loaded_model["freqs"]  # tải các thông số khác nếu cần

print("Model loaded successfully.")
print(f"The loaded vector of weights is {[round(t, 8) for t in np.squeeze(theta)]}")


Model loaded successfully.
The loaded vector of weights is [3.1e-07, 0.00127481, -0.00110868]


In [51]:
# Feel free to change the tweet below
my_tweet = 'I am very disappointed with the product quality. It broke after just one use.'
print(process_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, freqs1, theta1)
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else:
    print('Negative sentiment')

['disappoint', 'product', 'qualiti', 'broke', 'one', 'use']
[[0.4974998]]
Negative sentiment
